Тональность отзывов на основе Firefox browser add-ons review - Reviews for YouTube High Definition, отзывы с рейтингами 1 и 5
https://addons.mozilla.org/en-US/firefox/addon/youtube-high-definition/reviews/

Для корректной работы мне пришлось установить через терминал следующие библиотеки и модели
python3 -m spacy download en_core_web_sm
pip install spaCy
pip install langid
pip install response

In [94]:
import requests
import random
from bs4 import BeautifulSoup
import time

In [46]:
# скачиваем в список page с сайта url r страниц
def upload(url, r):
    session = requests.session()
    page = []
    for i in range(2, r):
        link = url +str(i)
        time.sleep(random.uniform(0.1, 0.2))
        req = session.get(link)
        page.append(req.text)
    return(page)

In [45]:
rating1_url = 'https://addons.mozilla.org/en-US/firefox/addon/youtube-high-definition/reviews/?score=1&utm_content=featured&utm_medium=referral&utm_source=addons.mozilla.org&page='
rating5_url = 'https://addons.mozilla.org/en-US/firefox/addon/youtube-high-definition/reviews/?score=5&utm_content=featured&utm_medium=referral&utm_source=addons.mozilla.org&page='

In [83]:
# скачиваем отзывы с рейтингом 1 и 5, 6 и 26 страниц соответственно
rating1 = upload(rating1_url, 8)
rating5 = upload(rating5_url, 28)

У отзывов с рейтингом 5 очень мало текстовых пояснений, кроме того, они написаны на разных языках, поэтому скачиваются все. Далее будет фильтрация отзывов на английском и выравнивание выборок.

In [79]:
# функция, вынимающая текст отзыва
def pars(ratings):
    text = []
    for item in ratings:
        soup = BeautifulSoup(item, 'html.parser')
        content = soup.find_all('div', {'class': 'ShowMoreCard-contents'})
        l = len(content)
        for i in range(l):
            text.append(str(content[i])[40:-12]) #очищаем от тегов
    return(text)

In [498]:
rating5_text = pars(rating5)
rating1_text = pars(rating1)

Теперь у нас есть два списка отзывов, оба с большим количеством пустых элементов. Уберём их, они нам не нужны.

In [499]:
l = rating1_text.count('')
for i in range(l):
    rating1_text.remove('')
l = rating5_text.count('')
for i in range(l):
    rating5_text.remove('')
print(len(rating5_text))
print(len(rating1_text))

173
99


In [493]:
# Образец отзыва
print(rating1_text[10])

just breaks youtube. no videos play, all videos just go to a profile post about covid.


Теперь у нас есть два списка непустых отзывов, 173 положительных и 99 отрицательных, разница не слишком велика, поэтому для составления контрольных множеств используем оба списка полностью. Однако они на разных языках, поэтому нужно отобрать отзывы на английском как, во-первых, самые частотные, а во-вторых, пригодные для nltk и spaCy. Делать это будем с помощью библиотеки langid.

In [430]:
import langid
from nltk.tokenize import wordpunct_tokenize

In [431]:
# отбираем тексты на английском
def eng(text):
    text_eng = []
    for a_text in text:
        if langid.classify(a_text)[0] == 'en':
            text_eng.append(a_text)
    return(text_eng)

In [500]:
rating1_text = eng(rating1_text)
rating5_text = eng(rating5_text)
print(len(rating1_text), len(rating5_text))

82 122


In [356]:
# токенизируем тексты на английском: нижний регистр, без пунктуации, только слова
def token(text):
    text_preprocessed = []
    for a_text in text:
        a_tokens = [w.lower() for w in wordpunct_tokenize(a_text) if w.isalpha()]
        text_preprocessed.extend(a_tokens)
    return(text_preprocessed)

In [501]:
rating1_tokens=token(rating1_text)
rating5_tokens=token(rating5_text)
print(len(rating1_tokens), len(rating5_tokens))

2853 1162


In [502]:
# делаем тестовую выборку, по 10 отзывов с рейтингами 1 и 5
rating1_test = []
rating5_test = []
for i in range(10):
    rating1_test.append(rating1_text[72+i])
    rating5_test.append(rating5_text[112+i])

In [503]:
# убираем тестовые отзывы из основной выборки
rating1_text = rating1_text[:72]
rating5_text = rating5_text[:112]

In [208]:
#nltk лемматизирует довольно плохо, spacy справляется заметно лучше, будем использовать эту библиотеку
import spacy
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

In [282]:
from collections import defaultdict

In [505]:
#Лемматизируем списки слов
def lemma(tokens):
    lemmas = []
    for token in tokens:
        lemmas.append(nlp(token)[0].lemma_)
    return(lemmas)

In [313]:
# Фильтрация по частотности, создание множества лемм
def freq_filtre(lemmas, freq):
    d = defaultdict(int)
    res = set()
    for lemma in lemmas:
        d[lemma] += 1
    for key in d.keys():
        if d[key] > freq:
            res.add(key)
    return(res)

In [506]:
#формируем множества позитивных и негативных лемм, с фильтрацией по минимальной частотности от 1 до 3
rating1_lemmas=lemma(rating1_tokens)
rating5_lemmas=lemma(rating5_tokens)
negat = []
posit = []
for freq in range(3):
    r1 = freq_filtre(rating1_lemmas, freq)
    r5 = freq_filtre(rating5_lemmas, freq)
    negat.append(r1 - r5)
    posit.append(r5 - r1)

In [507]:

#Проверим, действительно ли что-то отфильтровалось
for i in range(3):
    print(len(negat[i]), len(posit[i]))

338 158
174 43
133 23


In [379]:
#Определяем тональность отзыва
def tone(review, negat, posit):
    res = ''
    # лемматизируем отзыв
    review_tokens = [w.lower() for w in wordpunct_tokenize(review) if w.isalpha()]
    review_lemmas = set((nlp(token)[0].lemma_ for token in review_tokens))
    if len(review_lemmas & negat) == 0 and len(review_lemmas & posit) == 0: # если в отзыве нет лемм из оценочных множеств, то и определить тональность нельзя
        res = 'not stated'
    elif len(review_lemmas & negat) == 0 or len(review_lemmas & posit)/len(review_lemmas & negat) > 1:
        res = 'positive'
    elif len(review_lemmas & posit) == 0 or len(review_lemmas & negat)/len(review_lemmas & posit) > 1:
        res = 'negative'
    else:
        res = 'not stated' # сюда попадают отзывы с равным объёмом пересечений с каждым оценочным множеством
    return(res)

In [508]:
#Проверим, верно ли работает функция
tone(rating1_text[69], negat[0], posit[0])

'negative'

In [465]:
#вычисляем переменные для расчёта accuracy. TP = pos(rating5_*), TN = neg(rating1_*), FP = pos(rating1_*), FN = neg(rating5_*)
def accuracy_var(text, negat, posit):
    neg = 0
    pos = 0
    ns = 0
    for review in text:
        res = tone(review, negat, posit)
        if res == 'negative':
            neg += 1
        elif res == 'positive':
            pos += 1
        elif res == 'not stated':
            ns += 1
    return(neg, pos, ns)

In [466]:
#считаем accuracy сетов с разной фильтрацией, на основной выборке. [0] = neg, [1] = pos, [0] = ns
for i in range(3):
    res1 = accuracy_var(rating1_text, negat[i], posit[i])
    res5 = accuracy_var(rating5_text, negat[i], posit[i])
    print('Filtre at frequency ', i+1)
    print('True positive=', res5[1], 'True negative=', res1[0], 'False positive', res1[1], 'False negative', res5[0], 'Not stated', res1[2]+res5[2])
    acc = (res1[0] + res5[1])/(res1[0]+res1[1]+res5[0]+res5[1])
    print('Accuracy=', acc) 

Filtre at frequency  1
True positive= 69 True negative= 68 False positive 0 False negative 0 Not stated 47
Accuracy= 1.0
Filtre at frequency  2
True positive= 53 True negative= 70 False positive 1 False negative 12 Not stated 48
Accuracy= 0.9044117647058824
Filtre at frequency  3
True positive= 49 True negative= 69 False positive 1 False negative 15 Not stated 50
Accuracy= 0.8805970149253731


In [464]:
#считаем accuracy сетов с разной фильтрацией, на тестовых данных
for i in range(3):
    res1 = accuracy_var(rating1_test, negat[i], posit[i])
    res5 = accuracy_var(rating5_test, negat[i], posit[i])
    print('Filtre at frequency ', i+1)
    print('True positive=', res5[1], 'True negative=', res1[0], 'False positive', res1[1], 'False negative', res5[0], 'Not stated', res1[2]+res5[2])
    acc = (res1[0] + res5[1])/(res1[0]+res1[1]+res5[0]+res5[1])
    print('Accuracy=', acc) 

Filtre at frequency  1
True positive= 10 True negative= 9 False positive 0 False negative 0 Not stated 1
Accuracy= 1.0
Filtre at frequency  2
True positive= 7 True negative= 7 False positive 0 False negative 0 Not stated 6
Accuracy= 1.0
Filtre at frequency  3
True positive= 7 True negative= 8 False positive 0 False negative 2 Not stated 3
Accuracy= 0.8823529411764706


Выброс слов с низкой частотностью снижает качество работы, особенно сильно растёт False negative. Это вполне ожидаемо, учитывая особенности отзывов: положительные отзывы намного короче отрицательных.

Как улучшить предложенный алгоритм?  
1. При оценке тональности сейчас достаточно, чтобы объём пересечений с одним из оценочных множеств был больше:
    elif len(review_lemmas & negat) == 0 or len(review_lemmas & posit)/len(review_lemmas & negat) > 1:
        res = 'positive'
    elif len(review_lemmas & posit) == 0 or len(review_lemmas & negat)/len(review_lemmas & posit) > 1:
        res = 'negative'
Но у нас явный перекос в сторону отрицательных элементов, 338:158, и при равных весах часто возникает False negative. Поэтому можно было бы дать положительным леммам больший вес, признавать отзыв негативным при перевесе, например, в 338/158, т.е. примерно 2.14 раза, и наоборот для положительных: 158/338, примерно 0,5

In [476]:
def tone_normalized(review, negat, posit):
    res = ''
    # лемматизируем отзыв
    review_tokens = [w.lower() for w in wordpunct_tokenize(review) if w.isalpha()]
    review_lemmas = set((nlp(token)[0].lemma_ for token in review_tokens))
    if len(review_lemmas & negat) == 0 and len(review_lemmas & posit) == 0: # если в отзыве нет лемм из оценочных множеств, то и определить тональность нельзя
        res = 'not stated'
    elif len(review_lemmas & negat) == 0 or len(review_lemmas & posit)/len(review_lemmas & negat) > 0.5:
        res = 'positive'
    elif len(review_lemmas & posit) == 0 or len(review_lemmas & negat)/len(review_lemmas & posit) > 2.14:
        res = 'negative'
    else:
        res = 'not stated' # сюда попадают отзывы с равным объёмом пересечений с каждым оценочным множеством
    return(res)

In [468]:
def accuracy_var_normalized(text, negat, posit):
    neg = 0
    pos = 0
    ns = 0
    for review in text:
        res = tone_normalized(review, negat, posit)
        if res == 'negative':
            neg += 1
        elif res == 'positive':
            pos += 1
        elif res == 'not stated':
            ns += 1
    return(neg, pos, ns)

In [477]:
res1 = accuracy_var_normalized(rating1_text, negat[2], posit[2])
res5 = accuracy_var_normalized(rating5_text, negat[2], posit[2])
print('True positive=', res5[1], 'True negative=', res1[0], 'False positive', res1[1], 'False negative', res5[0], 'Not stated', res1[2]+res5[2])
acc = (res1[0] + res5[1])/(res1[0]+res1[1]+res5[0]+res5[1])
print('Accuracy=', acc) 

True positive= 56 True negative= 67 False positive 2 False negative 11 Not stated 48
Accuracy= 0.9044117647058824


Результат при равных весах:
Filtre at frequency  3
True positive= 49 True negative= 69 False positive 1 False negative 15 Not stated 50
Accuracy= 0.8805970149253731

2. Для коррекции перекоса в сторону отрицательных отзывов можно также добавить в множество положительной оценки частотные слова положительных отзывов (за исключением стоп-слов), которые встречаются и в отрицательных отзывах. У меня слишком мало времени до дедлайна, чтобы сделать всё красиво, поэтому я добавлю несколько самых частотных слов вручную

In [ ]:

d = defaultdict(int)
for lemma in rating5_lemmas:
    d[lemma] += 1
for key in sorted(d, key=d.get, reverse=True):
    print(key, d[key])

work good like use ok

In [513]:
for item in ['work', 'good', 'like', 'use', 'ok']:
    posit[2].add(item)

In [516]:
res1 = accuracy_var(rating1_text, negat[2], posit[2])
res5 = accuracy_var(rating5_text, negat[2], posit[2])
print('True positive=', res5[1], 'True negative=', res1[0], 'False positive', res1[1], 'False negative', res5[0], 'Not stated', res1[2]+res5[2])
acc = (res1[0] + res5[1])/(res1[0]+res1[1]+res5[0]+res5[1])
print('Accuracy=', acc) 

True positive= 68 True negative= 64 False positive 4 False negative 12 Not stated 36
Accuracy= 0.8918918918918919


Результат с исходным множеством позитивных токенов:
Filtre at frequency  3
True positive= 49 True negative= 69 False positive 1 False negative 15 Not stated 50
Accuracy= 0.8805970149253731